# This uses the HostFactory REST API to solicit machines

In [ ]:
from requests.auth import HTTPBasicAuth
import datetime
import json

from importlib import reload

import src.hostfactory
reload(src.hostfactory)

from src.hostfactory.api import HostFactory
from src.hostfactory.models import (
    HFRequestStatus,
    HFRequest,
    HFRequest_demand_hosts,
    HFDeleteRequest,
    HFDeleteRequest_request,
    HFRequestInstances,
    HFDeleteRequestInstances,
    HFCloudHosts,
    HFDeleteCloudHosts,
    HFCloudRequests,
    HFServiceConfiguration
)

from src.ego.api import Ego

# egosh client view REST_HOST_FACTORY_URL // default = :9080/platform/rest/hostfactory
# egosh client view REST_BASE_URL_1 // default = :8180/platform/rest/ego and :8180/platform/rest/deployment
# This api can be explored by using the following URL: http://example.host:8180/cloud/apis/explorer/#/

auth = HTTPBasicAuth(username="Admin", password="Admin")

hf = HostFactory(
    base_url="http://127.0.0.1:9080/platform/rest/hostfactory",
    auth=auth,
)

ego = Ego(
    base_url="http://127.0.0.1:8180/platform/rest/ego",
    auth=auth
)

pretty_print = lambda res: print(json.dumps(res.json(),indent=4))

print(
    "Current time is:",
    datetime.datetime.now().astimezone(tz=datetime.timezone.utc).isoformat(timespec="seconds")
)

### Get available provider instances

In [ ]:
res = hf.get_provider_instances()
res.json()

### Get templates

In [ ]:

# res = hf.get_host_templates(provider="gcpgkeinst")
res = hf.get_host_templates(provider="gcpgceinst")
print(res.text)



### Upgrade batch limits to max

In [ ]:
res = hf.put_service_configuration(
    data = HFServiceConfiguration(
        HF_DEMAND_BATCH_LIMIT=20,
        HF_RETURN_BATCH_LIMIT=20
    )
)


assert res.ok, res.text

res = hf.get_service_configuration()
pretty_print(res)

### Create a new request

In [ ]:
TEST_ID = "test-gce-2"


# 100 k with ~5k nodes

request = HFRequest(
    request_name=TEST_ID,
    demand_hosts=[
        HFRequest_demand_hosts(
            prov_name="gcpgkeinst", template_name="cc-ssd-16", ninstances=2000,
        ),
        HFRequest_demand_hosts(
            prov_name="gcpgkeinst", template_name="cc-ssd-32", ninstances=1700,
        ),
        HFRequest_demand_hosts(
            prov_name="gcpgkeinst", template_name="cc-hd-16", ninstances=1300,
        ),
    ],
)

# 100k with < 5k nodes

# request = HFRequest(
#     request_name=TEST_ID,
#     demand_hosts=[
#         HFRequest_demand_hosts(
#             prov_name="gcpgkeinst", template_name="cc-ssd-32", ninstances=2000,
#         ),
#         HFRequest_demand_hosts(
#             prov_name="gcpgkeinst", template_name="cc-hd-32", ninstances=1200,
#         ),
#     ],
# )


# GCE

# request = HFRequest(
#     request_name=TEST_ID,
#     demand_hosts=[
#         HFRequest_demand_hosts(
#             prov_name="gcpgceinst", template_name="template-gcp-01", ninstances=3,
#         ),
#     ],
# )

request_response = hf.post_request(requestor="admin", data=request)
request_time = datetime.datetime.now().astimezone(tz=datetime.timezone.utc).isoformat(timespec="seconds")

assert request_response.ok, "Request failed..."

print(
    "Request time is:",
    request_time
)

pretty_print(request_response)

scheduledRequestId=request_response.json()["scheduled_request_id"][0]


### Get the request status

In [ ]:
request_instance = hf.get_request_instances(
    params=HFRequestInstances(
        scheduledRequestId=scheduledRequestId
    )
)

requestId = request_instance.json()["rows"][0]["requestId"]

request_instance.json()

In [ ]:
cloud_hosts = hf.get_cloud_hosts(
    HFCloudHosts(requestId=requestId)
)

pretty_print(cloud_hosts)

In [ ]:
cloud_hosts = hf.get_all_cloud_hosts(
    requestId=requestId
)

cloud_hosts.head()


In [ ]:
cloud_requests = hf.get_cloud_requests(
    params = HFCloudRequests(
        requestId = requestId,
    )
)

cloud_requests.json()

### Close the request

In [ ]:
delete_scheduled_requests = hf.delete_scheduled_requests(
    data=HFDeleteRequest(
        [
            HFDeleteRequest_request(
                id=id,
                close_instances=True,
            ) for id in request_response.json()["scheduled_request_id"]
        ]
    )
)

delete_time = datetime.datetime.now().astimezone(tz=datetime.timezone.utc).isoformat(timespec="seconds")

assert delete_scheduled_requests.ok, "Failed to delete scheduled request"

print(f"Delete time: {delete_time}")


### Query active scheduled requests

In [ ]:
res = hf.get_scheduled_requests(
    HFRequestStatus(
        # status="Active"
    )
)
scheduled_requests = res.json()
for row in res.json()["rows"]:
    print("===============================================================")
    print("Request ID:", row["requestId"])
    print("Request name:", row["requestName"])
    print("Create time:", datetime.datetime.fromtimestamp(row["createTime"]))
    print("Template:", row["templateOrUnit"])
    print("Quantity:", row["templateOrUnitQuantity"])


### Query arbitrary scheduled request id

In [ ]:
scheduled_request_id = "SD-4ebd0443-ef3f-4827-877b-258b8491a216"

In [ ]:

request_instances = hf.get_request_instances(
    params=HFRequestInstances(
        scheduledRequestId=scheduled_request_id
    )
)

scheduled_request = hf.get_scheduled_requests(
    HFRequestStatus(
        requestId=scheduled_request_id
    )
)
request_id = request_instances.json()["rows"][0]["requestId"]


cloud_requests = hf.get_cloud_requests(
    params=HFCloudRequests(
        requestId=request_id
    )
)


pretty_print(request_instances)
pretty_print(scheduled_request)
pretty_print(cloud_requests)

In [ ]:

cloud_hosts = hf.get_all_cloud_hosts(
    requestId=request_id
)

cloud_hosts.head()


In [ ]:
import os
from pathlib import Path

DATA_FOLDER = Path(os.getenv("WORKDIR")).joinpath("data")
test_folder = DATA_FOLDER.joinpath(TEST_ID)

if not os.path.isdir(test_folder):
    os.mkdir(test_folder)

with open(test_folder.joinpath("request_instances.json"), "w") as fh:
    json.dump(request_instances.json(), fh, indent=4)

with open(test_folder.joinpath("scheduled_request.json"), "w") as fh:
    json.dump(scheduled_request.json(), fh, indent=4)

with open(test_folder.joinpath("cloud_requests.json"), "w") as fh:
    json.dump(cloud_requests.json(), fh, indent=4)

cloud_hosts.to_csv(test_folder.joinpath("cloud_hosts.csv"))



### Delete said arbitrary scheduled request

In [ ]:
delete_scheduled_requests = hf.delete_scheduled_requests(
    data=HFDeleteRequest(
        [
            HFDeleteRequest_request(
                id=scheduled_request_id,
                close_instances=True,
            )
        ]
    )
)
delete_time = datetime.datetime.now().astimezone(tz=datetime.timezone.utc).isoformat(timespec="seconds")

assert delete_scheduled_requests.ok, "Failed to delete scheduled request"

print(f"Delete time: {delete_time}")

### Query arbitrary request id

In [ ]:
request_id = "d-1b0d7330-9e7d-4c8f-8bdf-ab332f13473b"

cloud_requests = hf.get_cloud_requests(
    params=HFCloudRequests(
        requestId=request_id
    )
)

cloud_hosts = hf.get_cloud_hosts(
    params=HFCloudRequests(
        requestId=request_id
    )
)

pretty_print(cloud_requests)
pretty_print(cloud_hosts)

# TESTS

In [ ]:
from collections import defaultdict

# ============== Scale Up ==============
TEST_ID="test-saturation-0"
prov_name="gcpgkeinst"
ninstances=5000
template_name="template-saturation"

iteration=1

results = defaultdict(dict)

request = HFRequest(
    request_name=f"{TEST_ID}-loop-{iteration}",
    demand_hosts=[
        HFRequest_demand_hosts(
            prov_name="gcpgkeinst", template_name="template-saturation", ninstances=ninstances,
        ),
    ],
)

results[iteration]["request"] = hf.post_request(requestor="admin", data=request)
results[iteration]["request_time"] = datetime.datetime.now().astimezone(tz=datetime.timezone.utc).isoformat(timespec="seconds")

assert results[iteration]["request"].ok, "Request failed..."

In [ ]:

# ============== Wait for provisioned ==============

from time import sleep

request_instances = None
while request_instances is None or request_instances.json()["rows"][0]["status"] != "Provisioned":
    request_instances = hf.get_request_instances(
    params=HFRequestInstances(
            scheduledRequestId=results[iteration]["request"].json()["scheduled_request_id"][0]
        )
    )
    requestId = request_instances.json()["rows"][0]["requestId"]
    print(f'Provisioned: {request_instances.json()["rows"][0]["quantityProvisioned"]}, Status: {request_instances.json()["rows"][0]["status"]}')    
    sleep(5)

# Wait for quantityProvisioned...



In [ ]:
request_instances = hf.get_request_instances(
params=HFRequestInstances(
        scheduledRequestId=results[iteration]["request"].json()["scheduled_request_id"][0]
    )
)

request_instances.json()

In [ ]:
# ============== Delete ==============

delete_scheduled_requests = hf.delete_scheduled_requests(
    data=HFDeleteRequest(
        [
            HFDeleteRequest_request(
                id=results[iteration]["request"].json()["scheduled_request_id"][0],
                close_instances=True,
            )
        ]
    )
)

results[iteration]["delete_request"] = delete_scheduled_requests
results[iteration]["delete_request_time"] = datetime.datetime.now().astimezone(tz=datetime.timezone.utc).isoformat(timespec="seconds")

assert delete_scheduled_requests.ok, "Failed to delete request"
